In [1]:
import functools

import jax
import numpy as np
import scanpy as sc

from cfp.metrics import compute_mean_metrics, compute_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp

In [2]:
split = 1

In [3]:
adata_preds = sc.read(f"/lustre/groups/ml01/workspace/ot_perturbation/models/cpa/combosciplex/adata_with_predictions_{split}.h5ad")

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [4]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_{split}.h5ad"

In [5]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [6]:
adata_ref_test = adata_test[adata_test.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_test, n_comps=10)

adata_ref_ood = adata_ood[adata_ood.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_ood, n_comps=10)

In [7]:
adata_pred_test = adata_preds[adata_preds.obs["split"]=="test"]
adata_pred_test.layers["CPA_PRED"] = adata_pred_test.obsm["CPA_pred"]
cfpp.project_pca(query_adata=adata_pred_test, ref_adata=adata_ref_test, layer="CPA_PRED")
cfpp.project_pca(query_adata=adata_test, ref_adata=adata_ref_test)
test_data_target_encoded = {}
test_data_target_decoded = {}
test_data_target_encoded_predicted = {}
test_data_target_decoded_predicted = {}
for cond in adata_test.obs["condition"].cat.categories:
    if cond == "control":
        continue
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm["X_pca"]
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X.toarray()
    test_data_target_decoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["CPA_pred"]
    test_data_target_encoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["X_pca"]

adata_pred_ood = adata_preds[adata_preds.obs["split"]=="ood"]
adata_pred_ood.layers["CPA_PRED"] = adata_pred_ood.obsm["CPA_pred"]
cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_ref_ood, layer="CPA_PRED")
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_ref_ood)
ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if cond == "control":
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm["X_pca"]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.toarray()
    ood_data_target_decoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["CPA_pred"]
    ood_data_target_encoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["X_pca"]

/tmp/ipykernel_345786/2350350615.py:2: ImplicitModificationWarning: Setting element `.layers['CPA_PRED']` of view, initializing view as actual.
  adata_pred_test.layers["CPA_PRED"] = adata_pred_test.obsm["CPA_pred"]
/tmp/ipykernel_345786/2350350615.py:18: ImplicitModificationWarning: Setting element `.layers['CPA_PRED']` of view, initializing view as actual.
  adata_pred_ood.layers["CPA_PRED"] = adata_pred_ood.obsm["CPA_pred"]


In [8]:
test_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in test_data_target_decoded_predicted.keys()
}
ood_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in ood_data_target_decoded_predicted.keys()
}

In [9]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]


ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)

test_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, test_data_target_decoded_predicted, test_deg_dict)
test_deg_target_decoded = jax.tree_util.tree_map(get_mask, test_data_target_decoded, test_deg_dict)


In [10]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")

deg_test_metrics = jax.tree_util.tree_map(compute_metrics, test_deg_target_decoded, test_deg_target_decoded_predicted)
deg_mean_test_metrics = compute_mean_metrics(deg_test_metrics, prefix="deg_test_")


In [11]:
test_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, test_data_target_encoded, test_data_target_encoded_predicted
)
mean_test_metrics_encoded = compute_mean_metrics(test_metrics_encoded, prefix="encoded_test_")

test_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics_fast, test_data_target_decoded, test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

In [12]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [13]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.7398407714707511,
 'decoded_ood_sinkhorn_div_1': 105.7384272984096,
 'decoded_ood_sinkhorn_div_10': 72.54913112095424,
 'decoded_ood_sinkhorn_div_100': 26.10324423653739,
 'decoded_ood_e_distance': 51.42301688229961,
 'decoded_ood_mmd': 0.2205053504024233}

In [14]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.746076412498951,
 'decoded_test_e_distance': 49.233251864895884,
 'decoded_test_mmd_distance': 0.22017040041585764}

In [15]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.7398407714707511,
 'decoded_ood_sinkhorn_div_1': 105.7384272984096,
 'decoded_ood_sinkhorn_div_10': 72.54913112095424,
 'decoded_ood_sinkhorn_div_100': 26.10324423653739,
 'decoded_ood_e_distance': 51.42301688229961,
 'decoded_ood_mmd': 0.2205053504024233}

In [16]:
mean_ood_metrics_encoded

{'encoded_ood_r_squared': -0.2162843176296779,
 'encoded_ood_sinkhorn_div_1': 29.539738178253174,
 'encoded_ood_sinkhorn_div_10': 26.342303139822825,
 'encoded_ood_sinkhorn_div_100': 25.135371889386857,
 'encoded_ood_e_distance': 49.98884853596899,
 'encoded_ood_mmd': 0.21340926630156382}

In [17]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.5329056126730782,
 'deg_ood_sinkhorn_div_1': 23.16515043803624,
 'deg_ood_sinkhorn_div_10': 15.110301085880824,
 'deg_ood_sinkhorn_div_100': 14.509485176631383,
 'deg_ood_e_distance': 28.89735474270682,
 'deg_ood_mmd': 0.30165902631623404}

In [18]:
deg_mean_test_metrics

{'deg_test_r_squared': 0.5675543497006098,
 'deg_test_sinkhorn_div_1': 23.5619731148084,
 'deg_test_sinkhorn_div_10': 15.128504951794943,
 'deg_test_sinkhorn_div_100': 14.074339946111044,
 'deg_test_e_distance': 27.99714012013961,
 'deg_test_mmd': 0.2857689211765925}

In [19]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.746076412498951,
 'decoded_test_e_distance': 49.233251864895884,
 'decoded_test_mmd_distance': 0.22017040041585764}

In [20]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.7398407714707511,
 'decoded_ood_sinkhorn_div_1': 105.7384272984096,
 'decoded_ood_sinkhorn_div_10': 72.54913112095424,
 'decoded_ood_sinkhorn_div_100': 26.10324423653739,
 'decoded_ood_e_distance': 51.42301688229961,
 'decoded_ood_mmd': 0.2205053504024233}

In [21]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.5329056126730782,
 'deg_ood_sinkhorn_div_1': 23.16515043803624,
 'deg_ood_sinkhorn_div_10': 15.110301085880824,
 'deg_ood_sinkhorn_div_100': 14.509485176631383,
 'deg_ood_e_distance': 28.89735474270682,
 'deg_ood_mmd': 0.30165902631623404}

In [22]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.746076412498951,
 'decoded_test_e_distance': 49.233251864895884,
 'decoded_test_mmd_distance': 0.22017040041585764}

In [23]:
mean_test_metrics_encoded

{'encoded_test_r_squared': -0.5765983506852547,
 'encoded_test_sinkhorn_div_1': 27.952562818924587,
 'encoded_test_sinkhorn_div_10': 24.321213881174724,
 'encoded_test_sinkhorn_div_100': 23.14941970507304,
 'encoded_test_e_distance': 46.049976458877346,
 'encoded_test_mmd': 0.21895436507960161}

In [24]:
ood_metrics_encoded

{'Givinostat+Dasatinib': {'r_squared': 0.7533559799194336,
  'sinkhorn_div_1': 4.976339817047119,
  'sinkhorn_div_10': 2.58270263671875,
  'sinkhorn_div_100': 1.8748140335083008,
  'e_distance': 3.5999318616791154,
  'mmd': 0.08188818},
 'Givinostat+SRT1720': {'r_squared': 0.805464506149292,
  'sinkhorn_div_1': 4.276900291442871,
  'sinkhorn_div_10': 2.0799732208251953,
  'sinkhorn_div_100': 1.435948371887207,
  'e_distance': 2.7346117732941995,
  'mmd': 0.07507881},
 'Panobinostat+Dasatinib': {'r_squared': -1.408726692199707,
  'sinkhorn_div_1': 59.019691467285156,
  'sinkhorn_div_10': 54.95061111450195,
  'sinkhorn_div_100': 53.29975128173828,
  'e_distance': 106.21404763208841,
  'mmd': 0.37372366},
 'Panobinostat+PCI-34051': {'r_squared': -1.411924123764038,
  'sinkhorn_div_1': 64.75074005126953,
  'sinkhorn_div_10': 60.18962860107422,
  'sinkhorn_div_100': 58.113433837890625,
  'e_distance': 115.7105106733564,
  'mmd': 0.38196754},
 'Panobinostat+SRT2104': {'r_squared': -1.6086711

In [25]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/results/cpa"

In [26]:
import os
import pandas as pd

pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, f"ood_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, f"ood_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_encoded).to_csv(os.path.join(output_dir, f"test_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_decoded).to_csv(os.path.join(output_dir, f"test_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(deg_test_metrics).to_csv(os.path.join(output_dir, f"test_metrics_deg_{split}.csv"))
pd.DataFrame.from_dict(deg_ood_metrics).to_csv(os.path.join(output_dir, f"ood_metrics_ood_{split}.csv"))
